<img src="Callysto_Notebook-Banner_Top_06.06.18.jpg"/>

In [ ]:
#Running this cell displays a button to toggle hidden code
#From: http://chris-said.io/2016/02/13/how-to-make-polished-jupyter-presentations-with-optional-code-visibility/

from IPython.display import HTML

HTML('''<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }
  
  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>''')

# Central Limit Theorem

This notebook contains several interactive widgets for understanding the central limit theorem.

**Please note: If you have hidden the code using the toggle in the cell above, the cells will appear as an empty line. However, they can still be run by pressing shift-enter with the empty cell selected.**

#### Run the cell below to set up a distribution of the relative probabilities for rolling different sides on a die:

In [ ]:
#This cell allows the user to set up the distribution:

sides = 0
side_values = []

#Asks the user to input a number from 1 - 10 that will be the number of sides the die has:
while sides < 1 or sides > 10:
    print("Please enter the number of sides on the die (1 - 10): ")
    sides = int(input())

#Asks the user to input values for each side of the die:
for n in range(1, sides+1):
    print(f"Please enter the value for side number {n}" + ": ")
    side = int(input())
    side_values.append(side)

print()
print("The values you have chosen are:")
print(side_values)
print()
print("Please enter the probability of landing on each side. The total must add to 1 (100%).")
print()

#Asks the user to input values for each side of the die:
distribution = []
success = False

while success == False:

    for n in range(1, sides+1):
        print(f"Please enter the probability for side number {n}" + ": ")
        p = float(input())
        distribution.append(p)

    print()
    print("The probabilities you have chosen are:")
    print(distribution)
    
    #Checks if the input values add to 1, and asks the user to try again if not:
    #Round() is used to prevent float errors
    if round(sum(distribution), ndigits = 5) == 1:
        success = True
    else:
        print()
        print(f"The sum of your probabilities is {sum(distribution)}, but it must be equal to 1. Please re-enter your values.")
        distribution = []

# Run this cell below to display a graph of the distribution:
Note: In this plot and all others, the mean will be displayed as a vertical red line, and one standard deviation away from the mean on both sides will be displayed as a horizontal red line.

In [ ]:
#This cell displays a graph of the distribution:

#Setting up tools:
from IPython.display import display
from IPython.display import clear_output
import numpy as np
import pandas as pd
import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual
from ipywidgets import Button, HBox, VBox
import plotly
import plotly.offline as py
import plotly.graph_objs as go

py.init_notebook_mode(connected=False)
%matplotlib inline

total = sum(distribution)

#Converting the probability distribution into a pseudo-sample so that the histogram function can be used for the graph:
dist_data = []

for n in range(0, sides):  
    for m in range(0, int(distribution[n]*100)):
        dist_data.append(side_values[n])
        
#Making a histogram-type graph of the distribution:
data0 = [go.Histogram(x = dist_data, xbins=dict(start=min(side_values), end=max(side_values)+1, size=1))]
mean = sum(dist_data)/len(dist_data)
sd = np.std(dist_data)

layout = go.Layout(
        title="Probability Distribution (Mean: {0:.2f}, Standard deviation: {1:.2f})".format(mean, sd),
        xaxis=dict(
            title='Value',
            range = [min(side_values),max(side_values)+1],
            fixedrange = True,
            #tick0 = 0,
            #dtick = 1
        ),
        yaxis=dict(
            title='Percentage Probability',
            fixedrange = False
        ),
        shapes=[dict({
            'type': 'line',
            'x0': mean,
            'y0': 0,
            'x1': mean,
            'y1': 100,
            'line': {
                'color': '#f44242',
                'width': 3
        }}),
       dict({
        'type': 'line',
        'x0': mean - sd,
        'y0': 0,
        'x1': mean + sd,
        'y1': 0,
        'line': {
            'color': '#f44242',
            'width': 10
}})])

fig0 = go.Figure(data=data0, layout=layout)
py.iplot(fig0, filename='distribution')

#### Run this cell to take a sample of the distribution:

In [ ]:
#This cell displays a sample of the given distribution:

global distribution, probability, sample_size

#SAMPLE SIZE ENTRY BOX

ss_widget = widgets.BoundedIntText(
    value=1,
    min=1,
    max=10000000,
    step=1,
    description='Sample Size:',
    disabled=False
)

def ss_handler(w):
    global sample_size
    
    sample_size = ss_widget.value
    
    clear_output()
    display(ss_widget)
    graph_sample()
    

ss_widget.observe(ss_handler)

def create_sample():
    global distribution, probability, sample_size

    sample = []

    for n in range(0, sample_size):
        num = np.random.choice(side_values, p=distribution)
        sample.append(num)
        
    return sample

def graph_sample():
    global distribution, probability, sample_size
    
    total = sum(distribution)

    if total > 0:

        sample = create_sample()
        
        data = [go.Histogram(x = sample, xbins=dict(start=min(side_values), end=max(side_values)+1, size=1))]

        sample_mean = sum(sample)/len(sample)
        sd = np.std(sample)
        title = "Sampled Results (Mean: {0:.2f}, Standard deviation: {1:.2f}, Sample size: {2:.2f})".format(sample_mean, sd, sample_size)

        layout = go.Layout(
        title=title,
        xaxis=dict(
            title='Value',
            range = [min(side_values),max(side_values)+1],
            fixedrange = True,
            #tick0 = 0,
            #dtick = 1
        ),
        yaxis=dict(
            title='Count',
            fixedrange = False
        ),
        shapes=[dict({
                'type': 'line',
                'x0': sample_mean,
                'y0': 0,
                'x1': sample_mean,
                'y1': sample_size,
                'line': {
                    'color': '#f44242',
                    'width': 3
        }}),
               dict({
                'type': 'line',
                'x0': sample_mean - sd,
                'y0': 0,
                'x1': sample_mean + sd,
                'y1': 0,
                'line': {
                    'color': '#f44242',
                    'width': 10
}})])

        fig = go.Figure(data=data, layout=layout)
        py.iplot(fig, filename='Sample')

    #Exception for if all values are left at zero:
    else:
        print("All distribution values cannot be zero.")

display(ss_widget)

#### Run this cell to graph a number of sample means:

In [ ]:
# This cell displays a graph of the sample means:

#SAMPLE MEANS ENTRY BOX

sm_widget = widgets.BoundedIntText(
    value=0,
    min=0,
    max=100000,
    step=1,
    description='Samples:',
    disabled=False
)

def sm_handler(w):
    
    sample_means_num = sm_widget.value
    clear_output()
    display(sm_widget)
    graph_means(sample_means_num = sample_means_num)
    

sm_widget.observe(sm_handler)


def graph_means(sample_means_num):
    global distribution, probability, sample_size
    
    means_data = []
    temp_sample = []
    sample_size = ss_widget.value
    
    for n in range(0, sample_means_num):
        temp_sample = create_sample()
        mean = sum(temp_sample)/len(temp_sample)
        means_data.append(mean)

    data2 = [go.Histogram(x = means_data)]
    
    mean = np.mean(means_data)
    sd = np.std(means_data)

    title2 = "Sample Means (Mean: {0:.2f}, Standard deviation: {1:.2f})".format(mean, sd)

    layout2 = go.Layout(
    title=title2,
    xaxis=dict(
        title='Value',
        #range = [1,11],
        #fixedrange = True,
        #tick0 = 0,
        #dtick = 1
    ),
    yaxis=dict(
        title='Count',
        fixedrange = False
    
    ),
    shapes=[dict({
            'type': 'line',
            'x0': mean,
            'y0': 0,
            'x1': mean,
            'y1': ss_widget.value,
            'line': {
                'color': '#f44242',
                'width': 3
    }}),
           dict({
            'type': 'line',
            'x0': mean - sd,
            'y0': 0,
            'x1': mean + sd,
            'y1': 0,
            'line': {
                'color': '#f44242',
                'width': 10
}})])

    fig2 = go.Figure(data=data2, layout=layout2)
    py.iplot(fig2, filename='Means')


display(sm_widget)

<img src="Callysto_Notebook-Banners_Bottom_06.06.18.jpg"/>